# FRUIT SELLER APPLICATION

In [3]:
pip install prettytable

In [2]:
import sqlite3
from prettytable import PrettyTable

In [42]:
# connect to sql database 

db = sqlite3.connect("fruit_db.sqlite")
cursor = db.cursor()


# create table kept inside try and except to avoid requesting for creating duplicate table error
try:
    cursor.execute('''create table fruits
                  (   code integer,
                      name text,
                      price integer,
                      quantity integer      
                   ) ''')
except:
    pass


# adding new fruit row to table from user input
def add_item():
    code = int(input("Enter fruit code:"))
    name = input("Enter fruit name:")
    price = int(input("Enter fruit price:"))
    qnt = int(input("Enter fruit quantity:"))
    
    row = [code,name,price,qnt]
    cursor.execute("insert into fruits values(?,?,?,?)",row)
    db.commit()
    
    
# display the database 
def show_inventory():
    cursor.execute("select * from fruits")
    fruit_data = cursor.fetchall()
    t = PrettyTable(["CODE","NAME","PRICE/KG","QUANTITY"])
    for i in fruit_data:
        t.add_row(list(i))  # note we get a list of tuples so converting to a list(mutable)
    print(t)

    
# delete fruit row from database 
def delete_item():
    code = int(input('Enter fruit code to delete:'))
    cursor.execute('delete from fruits where code = ?',[code])
    db.commit()
    print('Data deleted sucessfully!')
    
    
def update_inventory(data,choices):
    for i in choices:
        code = data[i[0]][0]
        name = data[i[0]][1]
        qt_avail = data[i[0]][3]
        order = i[1]
        
        if qt_avail >= order :
            cursor.execute('update fruits set quantity = ? where code = ?',[qt_avail - order,code])
            db.commit()
        else:
            choices.remove(i)
            print('Not enough',name,'! Discarded from bill')
            
            
def print_bill(data,choices):
    print('******************FINAL BILL******************')
    update_inventory(data,choices)
    
    t = PrettyTable(["FRUIT","PRICE/KG","QUANTITY","TOTAL AMOUNT"])
    total_items = len(choices)
    total_amount = 0
    for i in choices:
        name = data[i[0]][1]
        prckg = data[i[0]][2]
        order = i[1]
        amt = order*prckg
        total_amount += amt
        l = [name,prckg,order,amt]
        t.add_row(l)

    print(t)
    print('Total Items:',total_items,'\n')
    print('Total amount:',total_amount)
    

# get the entire fruit data and codes
def get_fruit_data_codes():
    cursor.execute("select * from fruits")
    fruit_data = cursor.fetchall()
    codes = []
    for item in fruit_data:
        codes.append(item[0])
    return fruit_data,codes


# create bill for customer
def generate_bill():
    inp,sl = 0,0
    print('Press 1 to make bill\n')
    fruit_data,codes = get_fruit_data_codes()
    choices = []   # fruit_index,kg
    
    while True:
        sl += 1
        print(sl,'Enter code:')
        inp = int(input())

        if inp != 1:
            fruit_ind = codes.index(inp) 
            name = fruit_data[fruit_ind][1]
            print('Enter kg(',name,'):',sep='')
            kg = int(input())
            print(name,'Total kg:',kg,'\n')
            choices.append((fruit_ind,kg))
        else:
            print('Printing final bill....')
            print_bill(fruit_data,choices)
            break
        

# close the cursor and db connection
def close_db():
    cursor.close()
    db.close()
    
    
def inventory_manager():
    while True:
        t = PrettyTable(["MAIN MENU"])
        menu = [["1 New Bill"],["2 Add Item"],["3 Delete Item"],["4 Show Inventory"],["5 Shut Down"]]
        t.add_rows(menu)
        print(t)

        ch = int(input("Enter choice: "))
        if ch == 1:
            generate_bill()
        elif ch == 2:
            add_item()
        elif ch == 3:
            delete_item()
        elif ch == 4:
            show_inventory()
        elif ch == 5:
            print("Thank you visit again !")
            close_db()
            break


## ADDING NEW ITEM AND DISPLAY FROM DATABASE 

In [23]:
inventory_manager()

+------------------+
|    MAIN MENU     |
+------------------+
|    1 New Bill    |
|    2 Add Item    |
|  3 Delete Item   |
| 4 Show Inventory |
|   5 Shut Down    |
+------------------+
Enter choice: 2
Enter fruit code:101
Enter fruit name:apple
Enter fruit price:30
Enter fruit quantity:80
+------------------+
|    MAIN MENU     |
+------------------+
|    1 New Bill    |
|    2 Add Item    |
|  3 Delete Item   |
| 4 Show Inventory |
|   5 Shut Down    |
+------------------+
Enter choice: 4
+------+-------+----------+----------+
| CODE |  NAME | PRICE/KG | QUANTITY |
+------+-------+----------+----------+
| 101  | apple |    30    |    80    |
+------+-------+----------+----------+
+------------------+
|    MAIN MENU     |
+------------------+
|    1 New Bill    |
|    2 Add Item    |
|  3 Delete Item   |
| 4 Show Inventory |
|   5 Shut Down    |
+------------------+
Enter choice: 2
Enter fruit code:102
Enter fruit name:strawberry
Enter fruit price:50
Enter fruit quantity:102
+-----

## DELETE ITEM FROM DATABASE 

In [29]:
inventory_manager()

+------------------+
|    MAIN MENU     |
+------------------+
|    1 New Bill    |
|    2 Add Item    |
|  3 Delete Item   |
| 4 Show Inventory |
|   5 Shut Down    |
+------------------+
Enter choice: 2
Enter fruit code:105
Enter fruit name:litchi
Enter fruit price:35
Enter fruit quantity:56
+------------------+
|    MAIN MENU     |
+------------------+
|    1 New Bill    |
|    2 Add Item    |
|  3 Delete Item   |
| 4 Show Inventory |
|   5 Shut Down    |
+------------------+
Enter choice: 4
+------+------------+----------+----------+
| CODE |    NAME    | PRICE/KG | QUANTITY |
+------+------------+----------+----------+
| 101  |   apple    |    30    |    80    |
| 102  | strawberry |    50    |   102    |
| 103  |    kiwi    |    45    |    95    |
| 104  |   peach    |    35    |    89    |
| 105  |   litchi   |    35    |    56    |
+------+------------+----------+----------+
+------------------+
|    MAIN MENU     |
+------------------+
|    1 New Bill    |
|    2 Add Item    |

## GENERATING BILL FOR CUSTOMER 

In [44]:
inventory_manager()

+------------------+
|    MAIN MENU     |
+------------------+
|    1 New Bill    |
|    2 Add Item    |
|  3 Delete Item   |
| 4 Show Inventory |
|   5 Shut Down    |
+------------------+
Enter choice: 4
+------+------------+----------+----------+
| CODE |    NAME    | PRICE/KG | QUANTITY |
+------+------------+----------+----------+
| 101  |   apple    |    30    |    70    |
| 102  | strawberry |    50    |    77    |
| 103  |    kiwi    |    45    |    75    |
| 104  |   peach    |    35    |    89    |
+------+------------+----------+----------+
+------------------+
|    MAIN MENU     |
+------------------+
|    1 New Bill    |
|    2 Add Item    |
|  3 Delete Item   |
| 4 Show Inventory |
|   5 Shut Down    |
+------------------+
Enter choice: 1
Press 1 to make bill

1 Enter code:
101
Enter kg(apple):
10
apple Total kg: 10 

2 Enter code:
102
Enter kg(strawberry):
12
strawberry Total kg: 12 

3 Enter code:
103
Enter kg(kiwi):
80
kiwi Total kg: 80 

4 Enter code:
104
Enter kg(peac